In [0]:
### CODE TO ACCESS DRIVE ###
# Run
# Follow Link
# Copy Link, Paste and Enter 
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# Change to Drive folder - might be different path
%cd /content/drive/My Drive/'Colab Notebooks'/

/content/drive/My Drive/Colab Notebooks


In [0]:
# Check in correct folder 
%ls

checkpoint.pkl               Lab4.ipynb    UrbanSound8K_test.pkl
dataset.py                   logs/         UrbanSound8K_train.pkl
deep_learning_project.ipynb  __pycache__/


In [0]:
## Libraries
import time
from multiprocessing import cpu_count
from typing import Union, NamedTuple

import torch
import torch.backends.cudnn
import numpy as np
from torch import nn, optim
from torch.nn import functional as F
import torchvision.datasets
from torch.optim.optimizer import Optimizer
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from torchvision import transforms

import argparse
import os
from pathlib import Path

from torch.utils import data
from dataset import UrbanSound8KDataset
import random

torch.backends.cudnn.benchmark = True
parser = argparse.ArgumentParser(
                        formatter_class=argparse.ArgumentDefaultsHelpFormatter,
                        )
## Arguments
default_dataset_dir = Path.home() / ".cache" / "torch" / "datasets"
parser.add_argument("--log-dir", default=Path("logs"), type=Path)
parser.add_argument("--dataset-root", default=default_dataset_dir)
parser.add_argument("--learning_rate",default = 1e-3, type=float, help="Learning rate")
parser.add_argument("--sgd_momentum",default =  0.9, type=float)
parser.add_argument("--dropout", default = 0.5, type = float)
parser.add_argument("--mode", default = 'LMC', type = str)
parser.add_argument("-f", "--fff", help="a dummy argument to fool ipython", default="1")
parser.add_argument(
    "--batch-size",
    default=32,
    type=int,
    help="Number of images within each mini-batch",
)
parser.add_argument(
    "--epochs",
    default=31,
    type=int,
    help="Number of epochs (passes through the entire dataset) to train for",
)
parser.add_argument(
    "--val-frequency",
    default=1,
    type=int,
    help="How frequently to test the model on the validation set in number of epochs",
    )
parser.add_argument(
    "--log-frequency",
    default=10,
    type=int,
    help="How frequently to save logs to tensorboard in number of steps",
)
parser.add_argument(
    "--print-frequency",
    default=100,
    type=int,
    help="How frequently to print progress to the command line in number of steps",
)
parser.add_argument(
    "-j",
    "--worker-count",
    default=cpu_count(),
    type=int,
    help="Number of worker processes used to load data.",
)
parser.add_argument(
    "--checkpoint-path",
    default=Path("checkpoint.pkl"),
    type=Path,
    help="Provide a file to store checkpoints of the model parameters during training."
)
parser.add_argument(
    "--checkpoint-frequency",
    type=int, default=5,
    help="Save a checkpoint every N epochs"
)
parser.add_argument(
    "--weight_decay",
    default=1e-5,
    type=float,
    help="Weight decay: parameter related to L-2 regularisation.",
)

class ImageShape(NamedTuple):
    height: int
    width: int
    channels: int

if torch.cuda.is_available():
    DEVICE = torch.device("cuda")
else:
    DEVICE = torch.device("cpu")

def main(args):
    args.dataset_root.mkdir(parents=True, exist_ok=True)
    ## Choose which features to learn ( this can be an argument/ choose which net)
    mode = args.mode
    ## load data
    train_loader = torch.utils.data.DataLoader(
        UrbanSound8KDataset('UrbanSound8K_train.pkl', mode),
          batch_size=args.batch_size, shuffle=True,
          num_workers=args.worker_count, pin_memory=True
          )

    val_loader = torch.utils.data.DataLoader(
        UrbanSound8KDataset('UrbanSound8K_test.pkl', mode),
          batch_size=1, shuffle=False,
          num_workers=args.worker_count, pin_memory=True)
    height_val = 85
    if mode == 'MLMC': 
        height_val = 145
    model = CNN(height=height_val, width=41, channels=1, class_count=10, dropout = args.dropout, mode = args.mode)
    criterion = lambda logits, labels : nn.CrossEntropyLoss()(logits, labels)
    optimizer = torch.optim.Adam(model.parameters(), args.learning_rate, weight_decay=args.weight_decay)#, momentum =args.sgd_momentum)


    log_dir = get_summary_writer_log_dir(args)
    print(f"Writing logs to {log_dir}")
    summary_writer = SummaryWriter(
            str(log_dir),
            flush_secs=5
    )
    trainer = Trainer(
        model, train_loader, val_loader, criterion, optimizer, summary_writer, DEVICE, 
        args.checkpoint_path, checkpoint_frequency = args.checkpoint_frequency
    )

    trainer.train(
        args.epochs,
        args.val_frequency,
        print_frequency=args.print_frequency,
        log_frequency=args.log_frequency,
    )

    summary_writer.close()

class CNN(nn.Module):
    def __init__(self, height: int, width: int, channels: int, class_count: int, dropout: float, mode: str):
        super().__init__()
        self.input_shape = ImageShape(height=height, width=width, channels=channels)
        self.class_count = class_count
        ## Convolution layer 1
        self.conv1 = nn.Conv2d(
            in_channels=self.input_shape.channels,
            out_channels=32,
            kernel_size=(4,4),
            padding=(2,2),
            stride=(2,1),
        )
        # batch normalisation
        self.conv1_BN = nn.BatchNorm2d(32)
        self.initialise_layer(self.conv1)

        ## Convolution layer 2
        self.conv2 = nn.Conv2d(
            in_channels=self.conv1.out_channels,
            out_channels=64,
            kernel_size=(4,4),
            padding=(2,2),
            stride=(2,1),
            )
        # dropout
        self.dropout2d = nn.Dropout2d(dropout)
        self.conv2_BN = nn.BatchNorm2d(64)
        self.initialise_layer(self.conv2)
        # pooling layer
        self.pool = nn.MaxPool2d(kernel_size=(2,2), stride=(2,2), padding=(1,1))

        ## Convolution layer 3
        self.conv3 = nn.Conv2d(
            in_channels=self.conv2.out_channels,
            out_channels=128,
            kernel_size=(3,3),
            padding=(1,1),
            stride=(1,1),
        )
         
        self.conv3_BN = nn.BatchNorm2d(128)
        self.initialise_layer(self.conv3)
        ## Convolution layer 4
        self.conv4 = nn.Conv2d(
            in_channels=self.conv3.out_channels,
            out_channels=128,
            kernel_size=(3,3),
            padding=(1,1),
            stride=(1,1),
            )
        self.conv4_BN = nn.BatchNorm2d(128)
        self.initialise_layer(self.conv4)

        ## Fully-Connected layer
        in_size = 10752
        if mode == 'MLMC':
            in_size = 9216
        self.fc1 = nn.Linear(in_size, 1024)
        self.fc1_BN = nn.BatchNorm1d(1024)
        self.initialise_layer(self.fc1)
        self.dropout = nn.Dropout(dropout)
        ## Second Fully-Connected layer
       # self.fc2 = nn.Linear(8192, 1024)
        #self.fc2_BN = nn.BatchNorm1d(1024)
        #self.initialise_layer(self.fc2)

        ## Output layer
        self.fc3 = nn.Linear(1024, 10)
        self.initialise_layer(self.fc3)

    def forward(self, images: torch.Tensor) -> torch.Tensor:
        # Conv 1 --> batch norm --> relu
        x = F.relu(self.conv1_BN(self.conv1(images)))
        #print(x.size())
        # Conv 2 --> batch norm --> relu --> dropout --> pooling
        x = self.dropout2d(F.relu(self.conv2_BN(self.conv2(x))))
        #print(x.size())
        x = self.pool(x)
        #print(x.size())
        # Conv 3 --> batch norm --> relu --> dropout
        x = self.dropout2d(F.relu(self.conv3_BN(self.conv3(x))))
        #print(x.size())
        # Conv 4 --> batch norm --> relu --> dropout --> pooling
        x = self.dropout2d(F.relu(self.conv4_BN(self.conv4(x))))
        #print(x.size())
        x = self.pool(x)
        #print(x.size())
        # Flatten output of pooling layer
        x = torch.flatten(x, 1)
        # FC layer 1 --> batch norm --> sigmoid
        x = self.dropout(torch.sigmoid(self.fc1_BN(self.fc1(x))))
        #x = torch.sigmoid(self.fc1(x))
        # FC layer 2 -- sigmoid
        #x = torch.sigmoid(self.fc2_BN(self.fc2(x)))
        # Output layer
        x = self.fc3(x)
        #x = F.log_softmax(self.fc3(x), 1)
        return x

    @staticmethod
    def initialise_layer(layer):
        if hasattr(layer, "bias"):
            nn.init.zeros_(layer.bias)
        if hasattr(layer, "weight"):
            nn.init.kaiming_normal_(layer.weight)


class Trainer:
    def __init__(
        self,
        model: nn.Module,
        train_loader: DataLoader,
        val_loader: DataLoader,
        criterion: nn.Module,
        optimizer: Optimizer,
        summary_writer: SummaryWriter,
        device: torch.device,
        checkpoint_path: Path,
        checkpoint_frequency: int = 5
    ):
        self.model = model.to(device)
        self.device = device
        self.train_loader = train_loader
        self.val_loader = val_loader
        self.criterion = criterion
        self.optimizer = optimizer
        self.summary_writer = summary_writer
        self.step = 0
        self.checkpoint_path = checkpoint_path
        self.checkpoint_frequency = checkpoint_frequency

    def train(
        self,
        epochs: int,
        val_frequency: int,
        print_frequency: int = 20,
        log_frequency: int = 5,
        start_epoch: int = 0
    ):
        self.model.train()
        gamma = 0.55
        for epoch in range(start_epoch, epochs):
            self.model.train()
            data_load_start_time = time.time()
            for batch, labels, filename in self.train_loader:
                batch = batch.to(self.device)
                labels = labels.to(self.device)
                data_load_end_time = time.time()
                

                ##### Adding guassian noise ##### 
               # if epoch < 5: 
                    #Nu = random.randint(0, 1)
                    #if Nu == 0: 
                    #  Nu = 0.01
                    #elif Nu == 1: 
                    #  Nu = 0.5
                  #  Nu = 1
                 #   noise_var = (Nu/(1+self.step)**gamma)**0.5
                #    noise_torch = torch.randn(batch.size())
                   # noise_torch = noise_torch.to(self.device)
                   # batch += noise_torch

                # Forward pass
                logits = self.model.forward(batch)
                #break
                # Compute loss
                loss = self.criterion(logits, labels)
                # Compute the backward pass
                loss.backward()
                # Step optimizer
                self.optimizer.step()
                # Zero gradient buffers
                self.optimizer.zero_grad()
                # Compute accurracy
                with torch.no_grad():
                    preds = logits.argmax(-1)
                    accuracy, per_class_accuracy = compute_accuracy(labels, preds)

                data_load_time = data_load_end_time - data_load_start_time
                step_time = time.time() - data_load_end_time
                if ((self.step + 1) % log_frequency) == 0:
                    self.log_metrics(epoch, accuracy, loss, data_load_time, step_time)
                if ((self.step + 1) % print_frequency) == 0:
                    self.print_metrics(epoch, accuracy, loss, data_load_time, step_time)

                self.step += 1
                data_load_start_time = time.time()
            
            self.model_checkpoint(accuracy, epoch, epochs)

            self.summary_writer.add_scalar("epoch", epoch, self.step)
            if ((epoch + 1) % val_frequency) == 0:
                self.validate()
                # self.validate() will put the model in validation mode,
                # so we have to switch back to train mode afterwards
                self.model.train()

         # end validation code can go here
    def print_metrics(self, epoch, accuracy, loss, data_load_time, step_time):
        epoch_step = self.step % len(self.train_loader)
        print(
                f"epoch: [{epoch}], "
                f"step: [{epoch_step}/{len(self.train_loader)}], "
                f"batch loss: {loss:.5f}, "
                f"batch accuracy: {accuracy * 100:2.2f}, "
                f"data load time: "
                f"{data_load_time:.5f}, "
                f"step time: {step_time:.5f}"
        )

    def log_metrics(self, epoch, accuracy, loss, data_load_time, step_time):
        self.summary_writer.add_scalar("epoch", epoch, self.step)
        self.summary_writer.add_scalars(
                "accuracy",
                {"train": accuracy},
                self.step
        )
        self.summary_writer.add_scalars(
                "loss",
                {"train": float(loss.item())},
                self.step
        )
        self.summary_writer.add_scalar(
                "time/data", data_load_time, self.step
        )
        self.summary_writer.add_scalar(
                "time/data", step_time, self.step
        )

    def validate(self):
        results = {"preds": [], "labels": []}
        total_loss = 0
        self.model.eval()
        current_filename = ''
        file_scores = torch.zeros([1,10])
        file_scores = file_scores.to(self.device)

        # No need to track gradients for validation, we're not optimizing.
        with torch.no_grad():
            for batch, labels, filename in self.val_loader:
                if current_filename != filename and current_filename != '':
                    file_scores = F.softmax(file_scores, 1)
                    file_scores = torch.sum(file_scores, 0)/(file_scores.size()[0]-1)
                    preds = file_scores.argmax(dim=-1).cpu().numpy()
                    results["preds"].extend([preds])
                    results["labels"].extend(list(current_label.cpu().numpy()))
                    file_scores = torch.zeros([1,10])
                    file_scores = file_scores.to(self.device)
                    #print(current_filename)
                    

                batch = batch.to(self.device)
                labels = labels.to(self.device)
                logits = self.model(batch)
                loss = self.criterion(logits, labels)
                total_loss += loss.item()
                file_scores = torch.cat((file_scores, logits), 0)
                current_filename = filename
                current_label = labels


        accuracy, per_class_accuracy = compute_accuracy(
            np.array(results["labels"]), np.array(results["preds"])
        )
        average_loss = total_loss / len(self.val_loader)

        self.summary_writer.add_scalars(
                "accuracy",
                {"test": accuracy},
                self.step
        )
        self.summary_writer.add_scalars(
                "loss",
                {"test": average_loss},
                self.step
        )
        class_labels = ["ac", "ch", "cp", "db", "dr", "ei", "gs", "jh", "si", "sm"]
        print(f"validation loss: {average_loss:.5f}, accuracy: {accuracy * 100:2.2f}")
        for i in range(0,10):
            print(f"{class_labels[i]}_acc: {per_class_accuracy[i] * 100:2.2f}")

    def model_checkpoint(self, accuracy, epoch, epochs):
        if (epoch + 1) % self.checkpoint_frequency == 0 or (epoch + 1) == epochs:
            print(f"Saving model to {self.checkpoint_path}")
            torch.save({
                'epoch': epoch,
                'model': self.model.state_dict(),
                'accuracy': accuracy
            }, self.checkpoint_path)



def compute_accuracy(
    labels: Union[torch.Tensor, np.ndarray], preds: Union[torch.Tensor, np.ndarray]
) -> float:
    """
    Args:
        labels: ``(batch_size, class_count)`` tensor or array containing example labels
        preds: ``(batch_size, class_count)`` tensor or array containing model prediction
    """
    assert len(labels) == len(preds)
    class_acc = torch.zeros(10)
    for i in range(0,10):
        c = (preds[labels == i] == i).sum()
        d = sum(labels == i)
        if d != 0:
            class_acc[i] = c/d
        elif c > 0:
            class_acc[i] = 0
        else:
            class_acc[i] = 100

    return float((labels == preds).sum()) / len(labels), class_acc


def get_summary_writer_log_dir(args: argparse.Namespace) -> str:
    """Get a unique directory that hasn't been logged to before for use with a TB
    SummaryWriter.

    Args:
        args: CLI Arguments

    Returns:
        Subdirectory of log_dir with unique subdirectory name to prevent multiple runs
        from getting logged to the same TB log directory (which you can't easily
        untangle in TB).
    """
    tb_log_dir_prefix = (
      f"the_base_line"
      f"CNN_bn_"
      #f"dropout={args.dropout}_"
     # f"bs={args.batch_size}_"
      #f"lr={args.learning_rate}_"
      #f"momentum={args.sgd_momentum}_" +
      f"run_"
    )
    i = 0
    while i < 1000:
        tb_log_dir = args.log_dir / (tb_log_dir_prefix + str(i))
        if not tb_log_dir.exists():
            return str(tb_log_dir)
        i += 1
    return str(tb_log_dir)


if __name__ == "__main__":
    main(parser.parse_args())

Writing logs to logs/the_base_lineCNN_bn_run_145


RuntimeError: ignored